In [1]:
#just a bunch of fun
import numpy
import tensorflow as tf
import tensorflow_fold as td

# params
symbol='0123456789-,;.!?:’\”\"/\\|_@#$%^&*~`+ =<>()[]{}'
accent = 'áéíóöőúüű'
alphabet = 'abcdefghijklmnopqrstuvwxyz'
SPLIT_CHAR = '\t'

#our alphabet
hunabc = ' ' + alphabet + accent
hunabc += hunabc.upper()
vocabulary = sorted(set(hunabc))


In [2]:
def char_split(input_line, delimiter=''):
    source, target = tf.string_split(input_line, delimiter=delimiter)
    return source, target

def read_line(filename_queue):
    global SPLIT_CHAR
    reader = tf.TextLineReader(skip_header_lines=0)
    _, csv_row = reader.read(filename_queue)
    record_defaults = [[""], [""]]
    source, target = tf.decode_csv(csv_row, record_defaults=record_defaults, field_delim=SPLIT_CHAR)
    return source, target


In [3]:
filenames=["data.txt"]
filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
source, target = read_line(filename_queue)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
while True:
    try:
        sourcer, targetr = sess.run([source,target])
        print(sourcer, targetr)
    except tf.errors.OutOfRangeError:
        break
coord.request_stop()
coord.join(threads)

b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami cucc' b'negy valami cucc'
b'egy valami cucc' b'ket valami cucc'
b'harom valami